In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
print("Pandas version:", pd.__version__)

Pandas version: 2.2.3


In [2]:
df_Regions = pd.read_csv(r'/Users/yu-chenma/Documents/NCAAPrediction/Team Region Groups.csv')

In [3]:
df = pd.read_csv(r'/Users/yu-chenma/Documents/NCAAPrediction/games_2022.csv')

In [4]:
df = df.drop(columns=['home_away', 'tz_dif_H_E','notD1_incomplete'], axis=1)

In [5]:
df["pts_Differential"] = df["team_score"] - df["opponent_team_score"]

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
#win=1, lose = 0
df["Won"] = df.apply(lambda row: 1 if row["team_score"] > row["opponent_team_score"] 
                    else 0, axis=1)

In [8]:
#replaced nulls with 0s
df["OT_length_min_tot"].fillna(0, inplace=True)

In [9]:
df = df_Regions.merge(df, how="right", on="team")
#Add regions to the df

In [10]:
df["region"].fillna('East', inplace=True)

In [11]:
#group teams by region to make sure i didnt fuck up w the last step
teams_by_region = df.groupby("region")["team"].unique()

In [12]:
# Replacing Nulls values with 0 
# I do not know what to do with these values just yet so i just =0
df[["attendance", "rest_days","prev_game_dist","travel_dist","largest_lead"]] = df[["attendance", "rest_days","prev_game_dist","travel_dist","largest_lead"]].fillna(0)

In [13]:
df["cumulative_games_played"] = df.groupby("team").cumcount()
df["cumulative_Won"] = (df.groupby("team", group_keys=False)["Won"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["Winning_Percentage"] = (df["cumulative_Won"] / df["cumulative_games_played"]).fillna(0)

df["cumulative_BLK"] = (df.groupby("team", group_keys=False)["BLK"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_STL"] = (df.groupby("team", group_keys=False)["STL"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_TOV"] = (df.groupby("team", group_keys=False)["TOV"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_F_personal"] = (df.groupby("team", group_keys=False)["F_personal"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_AST"] = (df.groupby("team", group_keys=False)["AST"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_OREB"] = (df.groupby("team", group_keys=False)["OREB"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_DREB"] = (df.groupby("team", group_keys=False)["DREB"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
df["cumulative_DREB"] = (df.groupby("team", group_keys=False)["DREB"].apply(lambda x: x.shift(1).expanding().sum()).fillna(0))
#Reset the index again if needed
df = df.reset_index(drop=True)

In [14]:
#make copy of df that will represent the opponent stats.
df_opp = df.copy()

# all columns add an "opp_" prefix.
df_opp = df_opp.rename(columns=lambda x: x if x == "game_id" else "opp_team" if x == "team" else "opp_" + x)

# merge the original df with df_opp on the game_id.
df = pd.merge(df, df_opp, on="game_id", how="inner")

# remove  rows where the team is paired with itself.
df = df[df["team"] != df["opp_team"]]

#  Reset the index
df = df.reset_index(drop=True)

In [15]:
# Field Goal Percentage
df["fg_2_per"] = df.apply(lambda row: (row["FGM_2"] / row["FGA_2"]) if row["FGA_2"] > 0 else 0, axis=1)
df["fg_3_per"] = df.apply(lambda row: (row["FGM_3"] / row["FGA_3"]) if row["FGA_3"] > 0 else 0, axis=1)
df["ft_per"] = df.apply(lambda row: (row["FTM"] / row["FTA"]) if row["FTA"] > 0 else 0, axis=1)
#effective Field Goal Percentage
df["efg_per"] = df.apply(lambda row: (row["FGM_2"] + 1.5 * row["FGM_3"]) / (row["FGA_2"] + row["FGA_3"]) if (row["FGA_2"] + row["FGA_3"]) > 0 else 0, axis=1)

cumulative_cols = [
    "Won", "FGM_2", "FGM_3", "FGA_2", "FGA_3", "FTA", "FTM", "efg_per", "fg_3_per", "fg_2_per", 
    "largest_lead","team_score","opponent_team_score"]

# Apply fix to all cumulative features
for col in cumulative_cols:
    df[f"cumulative_{col}"] = (df.groupby("team", group_keys=False)[col]
                                .apply(lambda x: x.shift(1).expanding().sum())
                                .fillna(0))


# Calculate cumulative losses (using only past data)
df["cumulative_wins"] = df["cumulative_Won"]  # Won is already shifted
df["cumulative_losses"] = df["cumulative_games_played"] - df["cumulative_wins"]

# Calculate cumulative wins and games played for each tea
df["win_loss_ratio"] = df["cumulative_wins"] / df["cumulative_losses"].replace(0, np.nan)  # Replace 0 with NaN to avoid div by zero
df["win_loss_ratio"] = df["win_loss_ratio"].fillna(0)  # replace NaN cases with 0

df["win_per"] = df["cumulative_wins"] / df["cumulative_games_played"]
df["win_per"] = df["win_per"].fillna(0)  # replace NaN cases with 0


#3pt PAR(3pt attepmt rate)
df["3PAR"] = df.apply(lambda row: (row["FGA_3"]) / (row["FGA_2"] + row["FGA_3"]) if (row["FGA_2"] + row["FGA_3"]) > 0 else 0, axis=1)

#FTR(free throw rate)
df["FTR"] = df.apply(lambda row: (row["FTA"]) / (row["FGA_2"] + row["FGA_3"]) if (row["FGA_2"] + row["FGA_3"]) > 0 else 0, axis=1)

#Assist Turnover Ratio
df["ASTTO_ratio"] = df.apply(lambda row: (row["AST"] /(row["TOV"] + row["TOV_team"])) if (row["TOV"] + row["TOV_team"]) > 0 else 0, axis=1)

#True shooting percentage
df["TS_per"] = df.apply(lambda row: (row["team_score"] )/(2*((row["FGA_2"] + row["FGA_3"]) + 0.44 * row["FTA"])) if (2*((row["FGA_2"] + row["FGA_3"]) + 0.44 * row["FTA"])) > 0 else 0, axis=1)

In [16]:
redundant_columns = [
    'opp_game_date', 'opp_team_score', 'opp_opponent_team_score', 'opp_largest_lead', 
    'opp_OT_length_min_tot', 'opp_attendance',
    'opp_pts_Differential', 'opp_Won']
# Drop the redundant columns from the dataframe
df = df.drop(columns=redundant_columns)

In [17]:
# Create df_train and ensure no nulls in essential columns
df_train = pd.DataFrame()
df_train["team"] = df["team"]

df["game_id"] = df["game_id"].map(lambda x: x.lstrip("game_"))
df_train["game_id"]=df["game_id"]
train = df_train.pop("game_id")
df_train.insert(0, "game_id", train) #Move "game_id" to front

df_train["Region"] = df["region"]
train2 = df_train.pop("Region")
df_train.insert(2, "Region", train2) #Move "game_id" to front

df_train["game_date"] = df["game_date"]
train3 = df_train.pop("game_date")
df_train.insert(1, "game_date", train3) #Move "game_id" to front

# Avoid division by zero and handle NaNs
df_train["WinRatioPerGame"] = df["Winning_Percentage"]
df_train["PtsPerGame"] = (df["cumulative_team_score"] / df["cumulative_games_played"]).fillna(0)  # Points per game
df_train["PointRatio"] = (df_train["PtsPerGame"] / (df["cumulative_opponent_team_score"] / df["cumulative_games_played"])).fillna(0)  # Points versus opponent point ratio

df_train["FGPerGame"] = ((df["cumulative_FGM_2"] + df["cumulative_FGM_3"]) / df["cumulative_games_played"]).fillna(0)  # Shots made per game
df_train["FGAPerGame"] = ((df["cumulative_FGA_2"] + df["cumulative_FGA_3"]) / df["cumulative_games_played"]).fillna(0)  # Misses per game
df_train["FGRatio"] = (df_train["FGPerGame"] / df_train["FGAPerGame"]).fillna(0)  # Field goal percentage per game

df_train["FG_3_PerGame"] = (df["cumulative_FGM_3"] / df["cumulative_games_played"]).fillna(0)  # 3-pointer made per game
df_train["FGA_3_PerGame"] = (df["cumulative_FGA_3"] / df["cumulative_games_played"]).fillna(0)  # 3-pointer attempts per game
df_train["FG_3_Ratio"] = (df_train["FG_3_PerGame"] / df_train["FGA_3_PerGame"]).fillna(0)  # 3-pointer field goal percentage

df_train["FTMPerGame"] = (df["cumulative_FTM"] / df["cumulative_games_played"]).fillna(0)  # Free throw made per game
df_train["FTAPerGame"] = (df["cumulative_FTA"] / df["cumulative_games_played"]).fillna(0)  # Free throw attempt per game
df_train["FTRatio"] = (df_train["FTMPerGame"] / df_train["FTAPerGame"]).fillna(0)  # Free throw goal percentage

df_train["STLPerGame"] = (df["cumulative_STL"] / df["cumulative_games_played"]).fillna(0)  # Steals per game
df_train["TOVPerGame"] = (df["cumulative_TOV"] / df["cumulative_games_played"]).fillna(0)  # Turnovers per game
df_train["BLKPerGame"] = (df["cumulative_BLK"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["FoulPerGame"] = (df["cumulative_F_personal"] / df["cumulative_games_played"]).fillna(0)  # Fouls per game

df_train["FG_2_PPerGame"] = (df["cumulative_fg_2_per"] / df["cumulative_games_played"]).fillna(0)  # Field goal percentage per game (2pt)
df_train["FG_3_PPerGame"] = (df["cumulative_fg_3_per"] / df["cumulative_games_played"]).fillna(0)  # Field goal percentage per game (3pt)
df_train["eFGPerGame"] = (df["cumulative_efg_per"] / df["cumulative_games_played"]).fillna(0)  # Effective Field Goal Percentage per game (3pt) 

df_train["OREBinPossesions"] = (df["cumulative_OREB"] / (df["cumulative_OREB"] + df["opp_cumulative_DREB"])).fillna(0)  # Offensive rebound: Rebounds
df_train["DREBinPossesions"] = (df["cumulative_DREB"] / (df["cumulative_OREB"] + df["opp_cumulative_DREB"])).fillna(0)  # Defensive rebound: Rebounds

df_train["largestLeadPerGame"] = (df["cumulative_largest_lead"] / df["cumulative_games_played"]).fillna(0)  # Largest lead per game

df_train["OREBRatio"] = (df["cumulative_OREB"] / df["cumulative_games_played"]).fillna(0)  # Offensive rebound per game
df_train["DREBRatio"] = (df["cumulative_DREB"] / df["cumulative_games_played"]).fillna(0)  # Defensive rebound per game

df_train["BLKPerGame"] = (df["cumulative_BLK"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["FoulPerGame"] = (df["cumulative_F_personal"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["TOVPerGame"] = (df["cumulative_TOV"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["STLPerGame"] = (df["cumulative_STL"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game
df_train["ASTPerGame"] = (df["cumulative_AST"] / df["cumulative_games_played"]).fillna(0)  # Blocks per game

In [18]:
# Step 1: Shift 'Won' column by 1 for each team (ensuring only past data is used)
df["Won_Shifted"] = df.groupby("team")["Won"].shift(1).fillna(0)

# Step 2: Only use opponent's winning percentage **if the team has played at least one game**
df["PowerPoints"] = df.apply(lambda row: row["Winning_Percentage"] + 
                             (row["opp_Winning_Percentage"] if row["cumulative_games_played"] > 0  
                              else 0), axis=1)

# Step 3: Compute Cumulative Power Rank **only for teams that have played at least one game**
df["PowerRank"] = df.groupby("team")["PowerPoints"].cumsum().where(df["cumulative_games_played"] > 0, 0)

# Ensure Power Rank is in df_train
df_train["PowerRank"] = df["PowerRank"]

In [19]:
df_train.loc[df_train["team"]=="georgia_lady_bulldogs",["TOVPerGame"]].head(10)

,TOVPerGame
0,0.000000
11,18.000000
18,17.500000
33,18.000000
65,17.500000
71,16.000000
81,16.000000
103,16.285714
115,16.625000
123,17.111111


In [20]:
df.loc[df["team"]=="georgia_lady_bulldogs",["TOV", "cumulative_TOV", "cumulative_games_played"]].head(10)

,TOV,cumulative_TOV,cumulative_games_played
0,18,0.0,0
11,17,18.0,1
18,19,35.0,2
33,16,54.0,3
65,10,70.0,4
71,16,80.0,5
81,18,96.0,6
103,19,114.0,7
115,21,133.0,8
123,11,154.0,9


In [21]:
df["current_win_streak"] = 0
df["longest_win_streak"] = 0

# Compute win streak features per team
for team, group in df.groupby("team"):
    streak = 0
    longest_streak = 0
    streaks = []
    longest_streaks = []

    # Shift "Won" column to ensure we only use past data
    past_wins = group["Won"].shift(1).fillna(0)  # First game has no past info, so assume 0

    for won in past_wins:
        if won == 1:
            streak += 1
        else:
            streak = 0  # Reset streak
        
        longest_streak = max(longest_streak, streak)
        
        streaks.append(streak)
        longest_streaks.append(longest_streak)

    # Assign computed streaks back to df_train
    df.loc[group.index, "current_win_streak"] = streaks
    df.loc[group.index, "longest_win_streak"] = longest_streaks

In [22]:
df_train["current_win_streak"]=df["current_win_streak"]
df_train["longest_win_streak"]=df["longest_win_streak"]

In [23]:
#Spliting Teams
df_main_team = df_train.iloc[::2].reset_index(drop=True) 
df_opp_team = df_train.iloc[1::2].reset_index(drop=True)

df_main_team = pd.DataFrame(df_main_team)
df_opp_team = pd.DataFrame(df_opp_team)

In [24]:
df_train

,game_id,game_date,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,...,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
0,2022_2011,2021-12-30,georgia_lady_bulldogs,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
1,2022_2011,2021-12-30,lsu_tigers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
2,2022_2012,2021-12-30,missouri_tigers,South,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
3,2022_2012,2021-12-30,south_carolina_gamecocks,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
4,2022_2013,2021-12-30,tennessee_lady_volunteers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,2022_4795,2022-03-01,xavier_musketeers,East,0.310345,62.655172,0.860730,23.448276,59.344828,0.395119,...,0.415599,0.307897,0.629104,6.482759,11.965517,24.448276,12.793103,25.895827,0,3
10434,2022_4968,2022-03-04,harvard_crimson,East,0.480000,71.080000,1.048996,26.720000,67.960000,0.393172,...,0.459442,0.371459,0.589717,15.120000,14.160000,22.480000,15.240000,22.767969,0,4
10435,2022_4968,2022-03-04,dartmouth_big_green,East,0.120000,46.800000,0.728518,17.480000,55.000000,0.317818,...,0.368062,0.334911,0.708876,3.760000,11.320000,23.960000,9.240000,15.326778,0,1
10436,2022_5067,2022-03-06,harvard_crimson,East,0.500000,71.615385,1.066438,27.000000,68.000000,0.397059,...,0.464904,0.353398,0.572816,16.076923,14.000000,22.692308,15.423077,24.114123,1,4


In [25]:
df_train = df_train.rename_axis(None)  # Removes the index name

In [26]:
pd.set_option('display.max_columns', None)
df_train.describe() 

,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
count,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000,10438.000000
mean,0.461966,60.826877,0.954019,22.314880,55.810974,0.377920,5.537185,18.037765,0.287974,10.659931,15.152548,0.664695,7.418559,15.312767,3.016412,15.706252,0.421759,0.286497,0.425692,0.342109,0.769131,11.079940,10.944179,23.936511,12.241242,12.780660,1.397777,3.352079
std,0.278179,16.572495,0.288218,6.146264,14.005368,0.098799,2.203329,6.113402,0.084326,3.634984,4.970505,0.170307,2.597110,4.659895,1.447556,4.455020,0.112805,0.084280,0.112002,0.222950,0.544534,6.660954,3.583455,6.373803,3.907169,9.443290,2.417607,2.993575
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.260870,58.125000,0.875000,21.285714,55.545455,0.372593,4.400000,15.500000,0.269841,9.285714,13.305184,0.662059,6.300000,14.000000,2.142857,14.823529,0.412582,0.268745,0.417224,0.225970,0.510421,6.666667,9.571429,23.125000,10.880588,4.830296,0.000000,1.000000
50%,0.478261,63.711310,0.993452,23.240000,58.636364,0.397090,5.600000,18.431677,0.302673,11.000000,15.666667,0.700000,7.500000,15.740741,3.000000,16.300000,0.442074,0.301509,0.446881,0.308297,0.682406,10.555556,11.250000,25.000000,12.692308,11.490874,0.000000,3.000000
75%,0.666667,68.951786,1.102966,25.333333,61.500000,0.422256,6.833333,21.470588,0.330769,12.750000,18.027778,0.740000,8.881765,17.700000,3.900000,17.913043,0.473365,0.329109,0.477583,0.398113,0.880745,14.940257,13.000000,26.928571,14.432143,19.221275,2.000000,5.000000
max,1.000000,114.000000,2.435897,44.000000,82.000000,0.602941,19.000000,45.000000,0.727273,40.000000,46.000000,1.000000,20.000000,38.000000,13.000000,33.000000,0.695652,0.727273,0.669811,1.000000,8.333333,63.000000,27.000000,41.000000,31.000000,45.921987,28.000000,28.000000


In [27]:
#non-numerical columns
non_numeric_cols = ["game_id", "team", "Region","game_date"]

# Select only the numerical columns for subtraction
numeric_cols = [col for col in df_main_team.columns if col not in non_numeric_cols]

df_main_diff = df_main_team.copy()  # retain non-numeric columns
df_main_diff[numeric_cols] = df_main_team[numeric_cols] - df_opp_team[numeric_cols] #subtract datafram

numeric_cols = [col for col in df_opp_team.columns if col not in non_numeric_cols]

#element-wise subtraction on numerical columns
df_opp_diff = df_opp_team.copy()  # Make a copy to retain non-numeric columns
df_opp_diff[numeric_cols] = df_opp_team[numeric_cols] - df_main_team[numeric_cols]

In [28]:
df_train_final = pd.concat([df_main_diff, df_opp_diff], ignore_index=True) # concat

In [29]:
df_train_final['sort_index'] = np.arange(len(df_train_final)) % (len(df_train_final) // 2) * 2 + (np.arange(len(df_train_final)) // (len(df_train_final) // 2))

# Sort based on the new index and drop it afterward
df_train_final = df_train_final.sort_values(by='sort_index').drop(columns=['sort_index']).reset_index(drop=True)
df_train_final["Venue"] = df["home_away_NS"]
df_train_final["Results"] = df["Won"]

In [30]:
df_train_final

,game_id,game_date,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak,Venue,Results
0,2022_2011,2021-12-30,georgia_lady_bulldogs,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,0
1,2022_2011,2021-12-30,lsu_tigers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,-1,1
2,2022_2012,2021-12-30,missouri_tigers,South,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,1
3,2022_2012,2021-12-30,south_carolina_gamecocks,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,-1,0
4,2022_2013,2021-12-30,tennessee_lady_volunteers,North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,2022_4795,2022-03-01,xavier_musketeers,East,-0.289655,-6.478161,-0.149992,-2.585057,-2.521839,-0.025678,-4.375862,-10.356322,-0.079334,3.067816,4.582759,-0.027048,0.187356,3.686207,-0.467816,3.558621,-0.041020,-0.073476,-0.061583,0.032105,-0.167628,-4.817241,2.965517,-1.551724,-2.306897,-1.842330,-1,-3,1,0
10434,2022_4968,2022-03-04,harvard_crimson,East,0.360000,24.280000,0.320478,9.240000,12.960000,0.075354,3.640000,6.320000,0.074578,2.160000,2.640000,0.037194,4.360000,-5.520000,0.400000,1.360000,0.076909,0.076272,0.091381,0.036547,-0.119159,11.360000,2.840000,-1.480000,6.000000,7.441192,0,3,-1,1
10435,2022_4968,2022-03-04,dartmouth_big_green,East,-0.360000,-24.280000,-0.320478,-9.240000,-12.960000,-0.075354,-3.640000,-6.320000,-0.074578,-2.160000,-2.640000,-0.037194,-4.360000,5.520000,-0.400000,-1.360000,-0.076909,-0.076272,-0.091381,-0.036547,0.119159,-11.360000,-2.840000,1.480000,-6.000000,-7.441192,0,-3,1,0
10436,2022_5067,2022-03-06,harvard_crimson,East,-0.346154,2.884615,-0.275154,1.384615,5.692308,-0.014052,2.884615,9.500000,-0.017302,-2.769231,-2.230769,-0.086515,0.538462,-0.846154,0.038462,1.115385,0.006398,-0.024803,0.000300,-0.027504,-0.126030,-7.192308,0.038462,-2.923077,1.692308,-14.911202,-5,-8,1,0


In [31]:
home_win_rate = df_train_final[df_train_final["Venue"] == 1]["Results"].mean()
away_win_rate = df_train_final[df_train_final["Venue"] == -1]["Results"].mean()
neutral_win_rate = df_train_final[df_train_final["Venue"] == 0]["Results"].mean()

print(f"Home Win Rate: {home_win_rate:.2f}")
print(f"Away Win Rate: {away_win_rate:.2f}")
print(f"Neutral Win Rate: {neutral_win_rate:.2f}")

Home Win Rate: 0.59
Away Win Rate: 0.41
Neutral Win Rate: 0.50


In [32]:
#Linear realtionships with Results
nonnumeric = df_train_final[["team","game_id","Region","game_date"]]
#pd.set_option("display.max_rows", None) #Displays full column
df_train_final=df_train_final.drop(columns=nonnumeric)
#monotonic realtionships with Results
spearman = df_train_final.corr(method="spearman").iloc[-1]
spearman.sort_values(ascending = False)

Results               1.000000
PointRatio            0.479224
WinRatioPerGame       0.463672
largestLeadPerGame    0.448175
longest_win_streak    0.421080
FGPerGame             0.401180
PtsPerGame            0.394640
FGRatio               0.383485
eFGPerGame            0.368896
ASTPerGame            0.358451
PowerRank             0.354167
FG_2_PPerGame         0.348110
current_win_streak    0.321970
FG_3_Ratio            0.262385
FG_3_PPerGame         0.251436
DREBRatio             0.242697
OREBinPossesions      0.229423
FGAPerGame            0.226322
BLKPerGame            0.223965
DREBinPossesions      0.218891
FTMPerGame            0.212625
FG_3_PerGame          0.202235
FTRatio               0.185800
FTAPerGame            0.178205
Venue                 0.175287
STLPerGame            0.164698
OREBRatio             0.156574
FGA_3_PerGame         0.129010
FoulPerGame          -0.041563
TOVPerGame           -0.163203
Name: Results, dtype: float64

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler

x = df_train_final.drop(columns=["Results"], errors='ignore')
y = df_train_final["Results"].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)
print("Before Scaling - Train Min:", x_train.min().min(), "Max:", x_train.max().max())
print("Before Scaling - Test Min:", x_test.min().min(), "Max:", x_test.max().max())

# Min-max scaling (only using x_train values)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # Fit on train
x_test = scaler.transform(x_test)  # Transform test using train stats

Before Scaling - Train Min: -105.0 Max: 105.0
Before Scaling - Test Min: -85.0 Max: 87.0


In [34]:
print("After Scaling - Train Min:", x_train.min().min(), "Max:", x_train.max().max())
print("After Scaling - Test Min:", x_test.min().min(), "Max:", x_test.max().max())

After Scaling - Train Min: -7.2239336621667665 Max: 8.366934701089008
After Scaling - Test Min: -8.377560882385021 Max: 7.243136459833694


In [35]:
poly_svc = SVC(kernel='rbf', C=0.5, gamma=0.05994842503189409)
poly_svc.fit(x_train, y_train)

y_train_pred = poly_svc.predict(x_train)
y_pred = poly_svc.predict(x_test)
print('Training Accuracy:', accuracy_score(y_train, y_train_pred))
print('Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

Training Accuracy: 0.7516167664670659
Model accuracy score: 0.7375


In [36]:
df_train.columns

Index(['game_id', 'game_date', 'team', 'Region', 'WinRatioPerGame',
       'PtsPerGame', 'PointRatio', 'FGPerGame', 'FGAPerGame', 'FGRatio',
       'FG_3_PerGame', 'FGA_3_PerGame', 'FG_3_Ratio', 'FTMPerGame',
       'FTAPerGame', 'FTRatio', 'STLPerGame', 'TOVPerGame', 'BLKPerGame',
       'FoulPerGame', 'FG_2_PPerGame', 'FG_3_PPerGame', 'eFGPerGame',
       'OREBinPossesions', 'DREBinPossesions', 'largestLeadPerGame',
       'OREBRatio', 'DREBRatio', 'ASTPerGame', 'PowerRank',
       'current_win_streak', 'longest_win_streak'],
      dtype='object')

In [37]:
df_sorted = df_train.sort_values(by='game_date') 
last_game_stats = df_sorted.groupby('team').last().reset_index()
last_game_stats = last_game_stats[['game_id', 'game_date', 'team', 'Region', 'WinRatioPerGame',
       'PtsPerGame', 'PointRatio', 'FGPerGame', 'FGAPerGame', 'FGRatio',
       'FG_3_PerGame', 'FGA_3_PerGame', 'FG_3_Ratio', 'FTMPerGame',
       'FTAPerGame', 'FTRatio', 'STLPerGame', 'TOVPerGame', 'BLKPerGame',
       'FoulPerGame', 'FG_2_PPerGame', 'FG_3_PPerGame', 'eFGPerGame',
       'OREBinPossesions', 'DREBinPossesions', 'largestLeadPerGame',
       'OREBRatio', 'DREBRatio', 'ASTPerGame', 'PowerRank',
       'current_win_streak', 'longest_win_streak']]
last_game_stats = pd.DataFrame(last_game_stats)
#last_game_stats.drop(["game_id", "game_date"], axis=1, inplace=True)

In [38]:
last_game_stats

,game_id,game_date,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
0,2022_5129,2022-03-09,abilene_christian_wildcats,West,0.586207,74.000000,1.114805,25.689655,58.827586,0.436694,9.551724,26.827586,0.356041,13.068966,17.379310,0.751984,7.655172,14.482759,1.931034,16.172414,0.505279,0.354904,0.519726,0.307335,0.608171,17.413793,11.413793,22.586207,16.275862,28.776252,0,8
1,2022_841,2021-11-24,academy_of_art_urban_knights,East,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
2,2022_1701,2021-12-15,agnes_scott_college_scotties,East,0.000000,25.000000,0.213675,10.000000,51.000000,0.196078,3.000000,11.000000,0.272727,2.000000,4.000000,0.500000,7.000000,37.000000,2.000000,13.000000,0.175000,0.272727,0.225490,0.018152,0.016502,0.000000,11.000000,10.000000,6.000000,0.583333,0,0
3,2022_5114,2022-03-08,air_force_falcons,West,0.600000,62.466667,1.075775,23.433333,60.900000,0.384784,5.100000,17.000000,0.300000,10.500000,14.233333,0.737705,11.133333,13.333333,2.066667,17.633333,0.419133,0.288803,0.427457,0.305601,0.541186,11.800000,12.366667,21.900000,12.800000,31.445843,1,4
4,2022_5193,2022-03-11,akron_zips,South,0.629630,68.333333,1.033035,25.148148,57.851852,0.434699,6.185185,18.740741,0.330040,11.851852,17.148148,0.691145,7.148148,14.814815,3.592593,15.629630,0.488349,0.327462,0.490415,0.236958,0.618037,11.888889,9.925926,25.888889,14.703704,30.284491,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2022_1171,2021-12-01,xavier_la_gold_rush,East,0.000000,58.000000,0.920635,17.000000,53.000000,0.320755,4.000000,15.000000,0.266667,20.000000,22.000000,0.909091,8.000000,21.000000,1.000000,24.000000,0.342105,0.266667,0.358491,0.025090,0.046595,7.000000,14.000000,26.000000,13.000000,0.700000,0,0
560,2022_4946,2022-03-04,xavier_musketeers,East,0.333333,62.407407,0.863217,23.148148,58.740741,0.394073,2.740741,10.222222,0.268116,13.370370,18.000000,0.742798,7.592593,17.185185,2.962963,16.481481,0.416975,0.267368,0.414751,0.318049,0.645854,6.592593,12.074074,24.518519,12.962963,24.297387,0,3
561,2022_5173,2022-03-11,yale_bulldogs,East,0.600000,59.400000,1.042837,21.800000,56.720000,0.384344,6.320000,20.440000,0.309198,9.480000,13.920000,0.681034,7.160000,18.640000,3.680000,16.720000,0.426120,0.310107,0.440937,0.311408,0.691675,10.600000,12.120000,26.920000,13.920000,25.355923,0,3
562,2022_1127,2021-11-30,young_harris_college_mountain_lions,East,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0


In [39]:
East_Teams = last_game_stats[last_game_stats["Region"] == "East"]
West_Teams = last_game_stats[last_game_stats["Region"] == "West"]
North_Teams = last_game_stats[last_game_stats["Region"] == "North"]
South_Teams = last_game_stats[last_game_stats["Region"] == "South"]

Tourney_West = West_Teams.sort_values(by="PowerRank", ascending=False).head(16)
Tourney_East = East_Teams.sort_values(by="PowerRank", ascending=False).head(16)
Tourney_North = North_Teams.sort_values(by="PowerRank", ascending=False).head(16)
Tourney_South = South_Teams.sort_values(by="PowerRank", ascending=False).head(16)

In [40]:
#adding seeds
for df in [Tourney_East, Tourney_West, Tourney_North, Tourney_South]:
    df.insert(2, "Seed", range(1, len(df) + 1))

#Currently unsure about the seeds since ML thinks 16 is greater than 1

In [41]:
WR1_Home = Tourney_West.head(8)
WR1_Away = Tourney_West.tail(8).sort_values(by="Seed",ascending=False)

In [42]:
WR1_UpperBranch_Main = WR1_Home.iloc[[0,7,4,3]].reset_index(drop=True)
WR1_UpperBranch_Away = WR1_Away.iloc[[0,7,4,3]].reset_index(drop=True)

WR1_LowerBranch_Main = WR1_Home.iloc[[2,5,6,1]].reset_index(drop=True)
WR1_LowerBranch_Away = WR1_Away.iloc[[2,5,6,1]].reset_index(drop=True)

In [43]:
#WR1_UpperBranch Splitting
non_numeric_cols = ["game_id", "team", "Region","game_date","Seed"]

numeric_cols = [col for col in WR1_UpperBranch_Main.columns if col not in non_numeric_cols]

# subtract
WR1_UpperBranch_MainDiff = WR1_UpperBranch_Main.copy()
WR1_UpperBranch_AwayDiff = WR1_UpperBranch_Away.copy()

WR1_UpperBranch_MainDiff[numeric_cols] -= WR1_UpperBranch_Away[numeric_cols]
WR1_UpperBranch_AwayDiff[numeric_cols] -= WR1_UpperBranch_Main[numeric_cols]

# Concatenate results
WR1_UpperBranch_Diff = pd.concat([WR1_UpperBranch_MainDiff, WR1_UpperBranch_AwayDiff])
WR1_UpperBranch_Diff["Venue"] = 0
WR1_UpperBranch_Diff

#WR1_LowerBranch Splitting
numeric_cols = [col for col in WR1_LowerBranch_Main.columns if col not in non_numeric_cols]

# subtract
WR1_LowerBranch_MainDiff = WR1_LowerBranch_Main.copy()
WR1_LowerBranch_AwayDiff = WR1_LowerBranch_Away.copy()

WR1_LowerBranch_MainDiff[numeric_cols] -= WR1_LowerBranch_Away[numeric_cols]
WR1_LowerBranch_AwayDiff[numeric_cols] -= WR1_LowerBranch_Main[numeric_cols]

# Concatenate results
WR1_LowerBranch_Diff = pd.concat([WR1_LowerBranch_MainDiff, WR1_LowerBranch_AwayDiff])
WR1_LowerBranch_Diff["Venue"]=0
WR1_LowerBranch_Diff = WR1_LowerBranch_Diff.reset_index(drop=True)
WR1_UpperBranch_Diff = WR1_UpperBranch_Diff.reset_index(drop=True)

In [44]:
Round1Winner = WR1_UpperBranch_Diff[WR1_UpperBranch_Diff["Seed"].isin([1, 16, 8, 9, 5, 12, 4, 13])]

# For Sweet16Winner1:
pairings0 = [
    (Round1Winner.iloc[0]["team"], Round1Winner.iloc[4]["team"]), 
    (Round1Winner.iloc[1]["team"], Round1Winner.iloc[5]["team"]),
    (Round1Winner.iloc[2]["team"], Round1Winner.iloc[6]["team"]), 
    (Round1Winner.iloc[3]["team"], Round1Winner.iloc[7]["team"]),
]

Round1_pairings = pairings0

for matchup in Round1_pairings:
    print("Round 1 UpperBranch: " f"{matchup[0]} vs {matchup[1]}")

WR1_UpperBranch_Diff.drop(columns=["game_id", "Region", "game_date"], inplace=True)

# Initialize and train SVC
poly_svc = SVC(kernel='rbf', C=0.5, gamma=0.05994842503189409, probability=True)
poly_svc.fit(x_train, y_train)

# Select features directly from df_train_final
feature_cols = df_train_final.drop(columns=["Results"]).columns

# Apply the scaler transformation
X_tournament_scaled = scaler.transform(WR1_UpperBranch_Diff[feature_cols])

# Predict winners and probabilities in a single step
predictions, probabilities = poly_svc.predict(X_tournament_scaled), poly_svc.predict_proba(X_tournament_scaled)

# Add predictions and probabilities to the dataframe
WR1_UpperBranch_Diff["Predicted_Winner"] = predictions
WR1_UpperBranch_Diff["Winner_Confidence"] = probabilities[:, 1]
WR1_UpperBranch_Diff[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]

Round 1 UpperBranch: gonzaga_bulldogs vs ut_arlington_mavericks
Round 1 UpperBranch: california_baptist_lancers vs texas_longhorns
Round 1 UpperBranch: stanford_cardinal vs washington_state_cougars
Round 1 UpperBranch: oklahoma_sooners vs byu_cougars


,team,Predicted_Winner,Winner_Confidence,Seed
0,gonzaga_bulldogs,1,0.623703,1
1,california_baptist_lancers,0,0.257298,8
2,stanford_cardinal,1,0.784994,5
3,oklahoma_sooners,0,0.192709,4
4,ut_arlington_mavericks,0,0.305712,16
5,texas_longhorns,1,0.769317,9
6,washington_state_cougars,0,0.227458,12
7,byu_cougars,1,0.832419,13


In [45]:
#Print round 1 pairings
Round1LowerWinner = WR1_LowerBranch_Diff[WR1_LowerBranch_Diff["Seed"].isin([3,14,6,11,7,10,2,15])]

# For Sweet16Winner1:
pairingg = [
    (Round1LowerWinner.iloc[0]["team"], Round1LowerWinner.iloc[4]["team"]), 
    (Round1LowerWinner.iloc[1]["team"], Round1LowerWinner.iloc[5]["team"]),
    (Round1LowerWinner.iloc[2]["team"], Round1LowerWinner.iloc[6]["team"]), 
    (Round1LowerWinner.iloc[3]["team"], Round1LowerWinner.iloc[7]["team"]),
]

Round1Lower_pairings = pairingg

for matchup in Round1Lower_pairings:
    print("Round 1 LowerBranch: " f"{matchup[0]} vs {matchup[1]}")

X_tournament_scaled = scaler.transform(WR1_LowerBranch_Diff[feature_cols])

# Predicting winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)[:, 1]  # Confidence of winning class

#Add predictions and probabilities to the dataframe
WR1_LowerBranch_Diff["Predicted_Winner"] = predictions
WR1_LowerBranch_Diff["Winner_Confidence"] = probabilities
WR1_LowerBranch_Diff[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]

Round 1 LowerBranch: unlv_lady_rebels vs nebraska_cornhuskers
Round 1 LowerBranch: new_mexico_lobos vs baylor_bears
Round 1 LowerBranch: utah_utes vs arizona_wildcats
Round 1 LowerBranch: colorado_buffaloes vs long_beach_state_beach


,team,Predicted_Winner,Winner_Confidence,Seed
0,unlv_lady_rebels,0,0.405276,3
1,new_mexico_lobos,0,0.214347,6
2,utah_utes,0,0.450040,7
3,colorado_buffaloes,1,0.790892,2
4,nebraska_cornhuskers,1,0.559449,14
5,baylor_bears,1,0.777239,11
6,arizona_wildcats,1,0.616423,10
7,long_beach_state_beach,0,0.221597,15


In [46]:
# West Round 2 Upper Teams
WR2_UpperBranch = Tourney_West[Tourney_West["team"].isin(WR1_UpperBranch_Diff.loc[WR1_UpperBranch_Diff["Predicted_Winner"] == 1, "team"])]
WR2_UpperUpper_Main, WR2_UpperUpper_Away = WR2_UpperBranch.iloc[[0, 1]], WR2_UpperBranch.iloc[[2, 3]]
WR2_UpperUpper_Main, WR2_UpperUpper_Away = WR2_UpperUpper_Main.reset_index(drop=True), WR2_UpperUpper_Away.reset_index(drop=True)

# West Round 2 Lower Teams
WR2_LowerBranch = Tourney_West[Tourney_West["team"].isin(WR1_LowerBranch_Diff.loc[WR1_LowerBranch_Diff["Predicted_Winner"] == 1, "team"])]
WR2_LowerBranch = WR2_LowerBranch.reset_index(drop=True)
WR2_LowerLower_Main, WR2_LowerLower_Away = WR2_LowerBranch.iloc[[0, 2]], WR2_LowerBranch.iloc[[1, 3]]
WR2_LowerLower_Main, WR2_LowerLower_Away = WR2_LowerLower_Main.reset_index(drop=True), WR2_LowerLower_Away.reset_index(drop=True)

In [47]:
#WR2 Differences
non_numeric_cols = ["game_id", "team", "Region", "game_date", "Seed"]

def subtract_and_concat(main, away, numeric_cols):
    main_diff = main.copy()  
    away_diff = away.copy()
    main_diff[numeric_cols] = main[numeric_cols] - away[numeric_cols]  # Subtract the away DataFrame
    away_diff[numeric_cols] = away[numeric_cols] - main[numeric_cols]  # Subtract the main DataFrame
    return pd.concat([main_diff, away_diff])

numeric_cols_upper = [col for col in WR1_UpperBranch_Main.columns if col not in non_numeric_cols]
numeric_cols_lower = [col for col in WR1_LowerBranch_Main.columns if col not in non_numeric_cols]

# Upper Branch Difference
WR2_UpperBranch_Diff = subtract_and_concat(WR2_UpperUpper_Main, WR2_UpperUpper_Away, numeric_cols_upper)
WR2_UpperBranch_Diff["Venue"] = 0

# Lower Branch Difference
WR2_LowerBranch_Diff = subtract_and_concat(WR2_LowerLower_Main, WR2_LowerLower_Away, numeric_cols_lower)
WR2_LowerBranch_Diff["Venue"] = 0

In [61]:
#Print Round 2 Pairings
Sweet16Winner1 = WR2_UpperBranch_Diff[WR2_UpperBranch_Diff["Seed"].isin([1, 16, 8, 9, 5, 12, 4, 13])]

# For Sweet16Winner1:
pairings1 = [
    (Sweet16Winner1.iloc[0]["team"], Sweet16Winner1.iloc[2]["team"]),  # Winner of 1 vs 16 vs Winner of 8 vs 9
    (Sweet16Winner1.iloc[1]["team"], Sweet16Winner1.iloc[3]["team"]),  # Winner of 5 vs 12 vs Winner of 4 vs 13
]

sweet16_pairings = pairings1
for matchup in sweet16_pairings:
    print("Round 2 UpperBranch: " f"{matchup[0]} vs {matchup[1]}")

X_tournament_scaled = scaler.transform(WR2_UpperBranch_Diff[feature_cols])

# Predicting winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)[:, 1]  # Confidence of winning class

#Add predictions and probabilities to the dataframe
WR2_UpperBranch_Diff["Predicted_Winner"] = predictions
WR2_UpperBranch_Diff["Winner_Confidence"] = probabilities
WR2_UpperBranch_Diff[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]

Round 2 UpperBranch: gonzaga_bulldogs vs texas_longhorns
Round 2 UpperBranch: stanford_cardinal vs byu_cougars


,team,Predicted_Winner,Winner_Confidence,Seed
0,gonzaga_bulldogs,0,0.346017,1
1,stanford_cardinal,1,0.634554,5
0,texas_longhorns,1,0.630558,9
1,byu_cougars,0,0.358655,13


In [62]:
#Round 2 Prediction
WS16LowerLowerWinner = WR2_LowerBranch_Diff[WR2_LowerBranch_Diff["Seed"].isin([3,14,6,11,7,10,2,15])]

# For Sweet16Winner1:
pairings1 = [
    (WS16LowerLowerWinner.iloc[0]["team"], WS16LowerLowerWinner.iloc[2]["team"]),  # Winner of 1 vs 16 vs Winner of 8 vs 9
    (WS16LowerLowerWinner.iloc[1]["team"], WS16LowerLowerWinner.iloc[3]["team"]),  # Winner of 5 vs 12 vs Winner of 4 vs 13
]

WS16_LowerLower_pairings = pairings1
for matchup in WS16_LowerLower_pairings:
    print("Round 2 LowerBranch: " f"{matchup[0]} vs {matchup[1]}")

X_tournament_scaled = scaler.transform(WR2_LowerBranch_Diff[feature_cols])

# Predicting winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)[:, 1]  # Confidence of winning class

#Add predictions and probabilities to the dataframe
WR2_LowerBranch_Diff["Predicted_Winner"] = predictions
WR2_LowerBranch_Diff["Winner_Confidence"] = probabilities
WR2_LowerBranch_Diff[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]

Round 2 LowerBranch: colorado_buffaloes vs arizona_wildcats
Round 2 LowerBranch: baylor_bears vs nebraska_cornhuskers


,team,Predicted_Winner,Winner_Confidence,Seed
0,colorado_buffaloes,1,0.626812,2
1,baylor_bears,1,0.645072,11
0,arizona_wildcats,0,0.357171,10
1,nebraska_cornhuskers,0,0.321672,14


In [67]:
# WS16 Upper Teams
WS16_UpperBranch = Tourney_West[Tourney_West["team"].isin(WR2_UpperBranch_Diff.loc[WR2_UpperBranch_Diff["Predicted_Winner"] == 1, "team"])]
WS16_UpperBranch_Main, WS16_UpperBranch_Away = WS16_UpperBranch.iloc[[0]].reset_index(drop=True), WS16_UpperBranch.iloc[[1]].reset_index(drop=True)

# WS16 Lower Teams
WS16_LowerBranch = Tourney_West[Tourney_West["team"].isin(WR2_LowerBranch_Diff.loc[WR2_LowerBranch_Diff["Predicted_Winner"] == 1, "team"])]
WS16_LowerBranch_Main, WS16_LowerBranch_Away = WS16_LowerBranch.iloc[[0]].reset_index(drop=True), WS16_LowerBranch.iloc[[1]].reset_index(drop=True)

In [72]:
def calculate_diff_and_concat(main_df, away_df, non_numeric_cols):
    # Identify numeric columns by excluding non-numeric ones
    numeric_cols = [col for col in main_df.columns if col not in non_numeric_cols]

    # Calculate differences for both main and away
    main_diff = main_df.copy()
    away_diff = away_df.copy()

    main_diff[numeric_cols] = main_df[numeric_cols] - away_df[numeric_cols]  # Subtract away from main
    away_diff[numeric_cols] = away_df[numeric_cols] - main_df[numeric_cols]  # Subtract main from away

    # Concatenate results and add the Venue column
    diff_df = pd.concat([main_diff, away_diff])
    diff_df["Venue"] = 0

    return diff_df

# WS16 Upper Splitting
WS16_UpperBranch_Diff = calculate_diff_and_concat(WS16_UpperBranch_Main, WS16_UpperBranch_Away, non_numeric_cols)

# WS16 Lower Splitting
WS16_LowerBranch_Diff = calculate_diff_and_concat(WS16_LowerBranch_Main, WS16_LowerBranch_Away, non_numeric_cols)

In [75]:
#print WS16 pairings 
WS16_Winner = WS16_UpperBranch_Diff[WS16_UpperBranch_Diff["Seed"].isin([1, 16, 8, 9, 5, 12, 4, 13])]

pairings2 = [
    (WS16_Winner.iloc[0]["team"], WS16_Winner.iloc[1]["team"])
]
WS16_Winner = pairings2

for matchup in WS16_Winner:
    print("Sweet 16 UpperBranch: " f"{matchup[0]} vs {matchup[1]}")

X_tournament_scaled = scaler.transform(WS16_UpperBranch_Diff[feature_cols])

# Predicting winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)[:, 1] 

#Add predictions and probabilities to the dataframe
WS16_UpperBranch_Diff["Predicted_Winner"] = predictions
WS16_UpperBranch_Diff["Winner_Confidence"] = probabilities
WS16_UpperBranch_Diff[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]

Sweet 16 UpperBranch: stanford_cardinal vs texas_longhorns


,team,Predicted_Winner,Winner_Confidence,Seed
0,stanford_cardinal,1,0.774674,5
0,texas_longhorns,0,0.216132,9


In [76]:
#WS16 Lower Branch Prediction
WS16_Winner = WS16_LowerBranch_Diff[WS16_LowerBranch_Diff["Seed"].isin([3,14,6,11,7,10,2,15])]

pairings2 = [
    (WS16_Winner.iloc[0]["team"], WS16_Winner.iloc[1]["team"])
]
WS16_Winner = pairings2

for matchup in WS16_Winner:
    print("Sweet 16 LowerBranch: " f"{matchup[0]} vs {matchup[1]}")
    
X_tournament_scaled = scaler.transform(WS16_LowerBranch_Diff[feature_cols])

# Predicting winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)[:, 1] 

#Add predictions and probabilities to the dataframe
WS16_LowerBranch_Diff["Predicted_Winner"] = predictions
WS16_LowerBranch_Diff["Winner_Confidence"] = probabilities
WS16_LowerBranch_Diff[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]

Sweet 16 LowerBranch: colorado_buffaloes vs baylor_bears


,team,Predicted_Winner,Winner_Confidence,Seed
0,colorado_buffaloes,0,0.341830,2
0,baylor_bears,1,0.695405,11


In [77]:
# WE8 Main and Away
WE8_Main = Tourney_West[Tourney_West["team"].isin(WS16_UpperBranch_Diff.loc[WS16_UpperBranch_Diff["Predicted_Winner"] == 1, "team"])]
WE8_Away = Tourney_West[Tourney_West["team"].isin(WS16_LowerBranch_Diff.loc[WS16_LowerBranch_Diff["Predicted_Winner"] == 1, "team"])]
WE8_Main, WE8_Away = WE8_Main.reset_index(drop=True).iloc[[0]], WE8_Away.reset_index(drop=True).iloc[[0]]

In [80]:
numeric_cols = [col for col in WE8_Main.columns if col not in non_numeric_cols]

WE8_MainDiff = WE8_Main.copy()
WE8_AwayDiff = WE8_Away.copy()

# Subtract values from each other
WE8_MainDiff[numeric_cols] = WE8_Main[numeric_cols] - WE8_Away[numeric_cols]
WE8_AwayDiff[numeric_cols] = WE8_Away[numeric_cols] - WE8_Main[numeric_cols]

WE8_Diff = pd.concat([WE8_MainDiff, WE8_AwayDiff], ignore_index=True)
WE8_Diff["Venue"] = 0

In [82]:
#Printing WE8 pairings
WE8_Winner = WE8_Diff[WE8_Diff["Seed"].isin([3,14,6,11,7,10,2,15,1, 16, 8, 9, 5, 12, 4, 13])]

pairings2 = [
    (WE8_Winner.iloc[0]["team"], WE8_Winner.iloc[1]["team"])
]
WE8_Winner = pairings2

for matchup in WE8_Winner:
    print("Elite 8 UpperBranch: " f"{matchup[0]} vs {matchup[1]}")
X_tournament_scaled = scaler.transform(WE8_Diff[feature_cols])

# Predicting winners and probabilities
predictions = poly_svc.predict(X_tournament_scaled)
probabilities = poly_svc.predict_proba(X_tournament_scaled)[:, 1] 

#Add predictions and probabilities to the dataframe
WE8_Diff["Predicted_Winner"] = predictions
WE8_Diff["Winner_Confidence"] = probabilities
WE8_Diff[["team", "Predicted_Winner", "Winner_Confidence", "Seed"]]

Elite 8 UpperBranch: stanford_cardinal vs baylor_bears


,team,Predicted_Winner,Winner_Confidence,Seed
0,stanford_cardinal,1,0.760489,5
1,baylor_bears,0,0.234734,11


In [85]:
WF4_Winners = WE8_Diff.loc[WE8_Diff["Predicted_Winner"] == 1, "team"]
WF4 = Tourney_West[Tourney_West["team"].isin(WF4_Winners)].copy()
WF4

,game_id,game_date,Seed,team,Region,WinRatioPerGame,PtsPerGame,PointRatio,FGPerGame,FGAPerGame,FGRatio,FG_3_PerGame,FGA_3_PerGame,FG_3_Ratio,FTMPerGame,FTAPerGame,FTRatio,STLPerGame,TOVPerGame,BLKPerGame,FoulPerGame,FG_2_PPerGame,FG_3_PPerGame,eFGPerGame,OREBinPossesions,DREBinPossesions,largestLeadPerGame,OREBRatio,DREBRatio,ASTPerGame,PowerRank,current_win_streak,longest_win_streak
441,2022_5057,2022-03-06,5,stanford_cardinal,West,0.892857,73.785714,1.286426,27.642857,60.928571,0.453693,7.178571,19.964286,0.359571,11.321429,16.857143,0.67161,8.607143,13.785714,5.892857,16.107143,0.503807,0.362201,0.514603,0.345249,0.746002,21.214286,13.107143,28.321429,15.142857,39.131176,22,22
